In [8]:
! pip install -q kaggle

In [ ]:
#mount drive with colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir ~/.kaggle

In [11]:
!cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/kaggle.json

In [12]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sampadab17/network-intrusion-detection

In [ ]:
!unzip network-intrusion-detection.zip

In [15]:
import sklearn
import tensorflow as tf
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder


import warnings
warnings.filterwarnings('ignore')

In [16]:
df = pd.read_csv("/content/Train_data.csv")

In [ ]:
df.head(10)

In [ ]:
print(df.isnull().sum())

In [ ]:
print(df.dtypes)

In [ ]:
df['class'].value_counts()

In [21]:
def encoding(dataframe):
    for col in dataframe.columns:
        if dataframe[col].dtype == 'object':
                label_encoder = preprocessing.LabelEncoder()
                dataframe[col] = label_encoder.fit_transform(dataframe[col])

In [22]:
encoding(df)

In [ ]:
df.head(5)

In [ ]:
print(df.dtypes)

In [ ]:
df.shape

In [26]:
Y = df["class"].values
X = df.drop(labels = ["class"], axis=1)

In [ ]:
X.shape

In [ ]:
Y.shape

In [29]:
def my_model():
    model = Sequential()
    model.add(Dense(4, input_dim=41, activation='relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    return model

In [30]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [31]:
my_keras_model = KerasClassifier(build_fn=my_model, epochs=20, batch_size=50, verbose=0)

In [32]:
steps = list()
steps.append(('scaler', MinMaxScaler()))
steps.append(('model', my_keras_model))
pipeline = Pipeline(steps=steps)

In [33]:
cv = KFold(n_splits=8, random_state=42, shuffle=True)

In [ ]:
scores = cross_val_score(pipeline, X, Y, scoring='accuracy', cv=cv)

In [ ]:
for score in scores:
    print("Score for this split is: ", score)

In [ ]:
# report performance
print('Accuracy: ', (np.mean(scores)))

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [38]:
scaler = MinMaxScaler()

In [39]:
scaler.fit(X_train)
X_train = scaler.transform(X_train)  #scaler converts our dataframe into arrays
X_test = scaler.transform(X_test)

In [ ]:
model = my_model()
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train ,verbose=1, epochs=50, batch_size=32,
                     validation_data=(X_test, y_test))

In [ ]:
_, acc = model.evaluate(X_test, y_test)
print("Accuracy = ", (acc * 100.0), "%")

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Predicting the Test set results
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, cmap="YlGnBu")
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))